# PROYECTO 1 EDA - LIMPIEZA

In [1]:
import pandas as pd
import numpy as np
import src.clean as cl

### Cargar dataset

In [2]:
df_anime = pd.read_csv('../Proyecto1/data/anime-dataset-2023.csv')
df_anime.sample()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
13314,36066,The Sun Sank,The Sunk Sank,The Sunk Sank,5.26,UNKNOWN,"A music video for LITE's song ""The Sun Sank.""",Music,1.0,"Sep 17, 2009",...,UNKNOWN,Music,5 min,G - All Ages,UNKNOWN,17391,0,123.0,286,https://cdn.myanimelist.net/images/anime/11/87...


Comenzamos identificando la estructura del dataset previo a los cambios

### 1.2 Limpieza de datos

Vamos a comenzar analizando las columnas que tenemos en el dataset.

In [3]:
df_anime.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

Eliminamos columnas que no aportan información. Tanto el nombre en inglés como en japones no nos da información extra ya que tenemos la columna *name*. La sinopsis de la serie tampoco es importante para este tipo de análisis. Con la columan de *Studio* es más que suficiente y no necesitaremos *Producers* o *Licensors*. *Members* nos proporciona informacion sobre la cantidad de usuarios que han añadido la serie a sus listas, cosa que no nos aporta información extra teniendo otras columnas como la popularidad o los favoritos. La url de la imagen no podemos hacer nada con ella en este análisis.



In [4]:
#Para evitar modificar el original vamos a realizar una copia
df = df_anime.copy()
#Eliminamos las columnas mencionadas
columnas = ["English name", "Other name", "Synopsis", "Producers", "Licensors", "Duration", "Members", "Image URL"]
df = cl.drop_columns(df, columnas)
df.columns

Index(['anime_id', 'Name', 'Score', 'Genres', 'Type', 'Episodes', 'Aired',
       'Premiered', 'Status', 'Studios', 'Source', 'Rating', 'Rank',
       'Popularity', 'Favorites', 'Scored By'],
      dtype='object')

Vamos a normalizar el nombre de las columnas al tipo *snake_case*

In [5]:
df.columns = cl.normalizar_snake_case(df)
df.columns

Index(['anime_id', 'name', 'score', 'genres', 'type', 'episodes', 'aired',
       'premiered', 'status', 'studios', 'source', 'rating', 'rank',
       'popularity', 'favorites', 'scored_by'],
      dtype='object')

Vamos a convertir los valores nulos *UNKNOWN* y *Not available* en valores *NaN*

In [6]:
nulos = ["UNKNOWN", "Not available", "Unknown"]
df = cl.conversor_nulos(df, nulos)

#### Corregimos incoherencias y tipos de datos en las columnas

##### Columnas *anime_id* y *name*

Vamos a comenzar comprobando si existen nulos en columnas críticas como *anime_id* o *name*

In [7]:
# nulos_id = df["anime_id"].isnull().sum()
# nulos_name = df["name"].isnull().sum()
nulos_id = cl.comprobar_nulos(df, "anime_id")
nulos_name = cl.comprobar_nulos(df, "name")
print(f"Nulos en anime_id: {nulos_id}\nNulos en name: {nulos_name}")

Nulos en anime_id: 0
Nulos en name: 0


Visto que no tenemos ningún nulo pasamos a comprobar si existen valores duplicados. En estos datos no tendría que existieran valores repetidos. Para hacer una mejor comprobación vamos a convertir la columna *name* todo a minúsculas y sin espacios.

In [8]:
duplicados_id = cl.comprobar_duplicados(df, "anime_id")
duplicados_name = cl.comprobar_duplicados(df, "name", True)
print(duplicados_id)
print(duplicados_name)

0
12


Vemos como la columna *name* tiene varios duplicados. Vamos a eliminar el duplicado que tenga más *NaN* en la fila, para ello vamos a crear una columna extra auxiliar que nos indique el número total de nulos de la fila y de esta forma ordenar esta columna de menor a mayor para tener primero las filas con menos nulls. Además, nos hemos fijado que hay nombres duplicados pero escritos de distinta forma (mayusculas, sin espacios, etc.) por ese motivo tambien vamos a crear una columna auxiliar que sea la columna *name* pero todo en minusculas y sin espacios. La columna auxiliar *nulos_fila* la mantendremos para poder usarla más adelante.

In [9]:
df = cl.eliminar_duplicados_mas_nulos(df, "name")

Realizada la operación volvemos a comprobar si existen duplicados.

In [10]:
duplicados_name = cl.comprobar_duplicados(df, "name", True)

In [11]:
df.sample()

,anime_id,name,score,genres,type,episodes,aired,premiered,status,studios,source,rating,rank,popularity,favorites,scored_by,nulos_fila
18127,44457,Doby Disy: Ai Changge,NaN,NaN,TV,36.0,Dec 2015 to ?,NaN,Finished Airing,NaN,Original,PG - Children,15594.0,22424,0,NaN,5


##### Columna *score*

Convertimos el tipo de *score* a **float64** para que permita valores decimales y nulos

In [12]:
df["score"] = cl.convert_type(df, "score", "float64")
df['score'].dtype

dtype('float64')

##### Columna *type*

Comprobamos que hay filas con type Music. Para este analisis vamos a eliminar las filas que son videos musicales, únicamente nos quedaremos con series, películas o episodios especiales y spin-offs.

In [13]:
df = cl.filtrar_columna_type(df)

['TV' 'Movie' 'Special' 'ONA' 'OVA' 'Music' nan]


Convertimos los datos a tipos categóricos

In [14]:
df["type"] = cl.convert_type(df, "type", "category")

##### Columna *episodes*

Los datos de la columna *episodes* los convertimos a valores enteros. Necesitamos que sea **Int64** para que permita los *NaN*.

In [15]:
df['episodes'] = cl.convert_type(df, "episodes", "Float64")
df["episodes"] = cl.convert_type(df, "episodes", "Int64")

##### Columna *status*

Saltamos directamente a esta columna porque comprobamos como hay animes que aun no han sido emitidos (*Not yet aired*) y por tanto no son aptos para realizar un analisis. Procedemos a eliminar las filas con este valor. Ademas vamos a facilitar el uso de los datos transformando Finished Airing -> Finished  y Currently Airing -> Currently

In [16]:
df = cl.filtro_columna_status(df)

Valores de status antes del cambio: ['Finished Airing' 'Currently Airing' 'Not yet aired']
Valores de status tras el cambio: ['Finished' 'Currently']


Tras esto podemos convertir los valores a tipos categóricos

In [17]:
df['status'] = cl.convert_type(df, "status", "category")

##### Columnas *aired* y *premiered*

Para facilitar el uso de los datos de la columna *aired*, vamos a sustituirla por una nueva columna: *aired_start*. Además vemos innecesario mantener el día y mes al tener la columna *premiered* que nos indica en que temporada del año comenzó. Esta nueva columna *aired_start* nos indica el año en el que comenzó el anime. La columna *premiered* nos indica en que temporada del año se estrenó (invierno, primavera, verano y otoño).

Para este análisis no vamos a necesitar la fecha de fin del anime. No lo consideramos una variable importante al servir únicamente para comprobar cuanto tiempo a durado una serie en emisión. No es del todo fiable debido a que, en numerosas ocasiones, se producen pausas o retrasos en las emisiones, probocando que series que han podido tardar mucho más que otras, realmente tengan menos capítulos. Para realizar un análisis de duración utilizaremos mejor el número de capítulos.

In [18]:
df = cl.filtrar_columna_aired(df)

Inicialmente intentamos convertir la fecha al formato *date* pero se nos hizo imposible porque el dataset te incluía la fecha de muchas formas distintas: en la mayoría de datos si era un formato legible para el formato *date*, pero también hay filas donde únicamente sale el año, otras aparece la fecha de inicio y un ? para la fecha de fin, otras solo sale mes y año, etc. Por este motivo acabamos optando por hacer regex y poder obtener el año y mes de los valores que existan.

Para la columna *premierd* vamos a completar todos los valores *NaN* que podamos. Si conocemos el mes y año de inicio del anime podemos completar el valor de dicha fila. Vamos a utilizar la función *month_to_season(y, m)* para dado un año y mes en valor numérico te lo parse al formato requerido. Para comprabar su eficacia analizamos antes y depsués el numero de nulos en la columna.

In [19]:
df.shape

(21724, 19)

In [20]:
cl.comprobar_nulos(df, "premiered")

16322

Vemos que la columna contiene 16322 nulos de 21724 filas antes de realizar la operación. Veamos cuantos logramos eliminar.

Buscamos todas las filas donde *premiered* sean *NaN* y además necesitamos que la columna *aired_start* con el año de inicio y la columna auxiliar *aired_month* no sean nulas. Una vez filtrado aplicamos la función a todas las filas.

In [21]:
df = cl.completar_premiered(df)

Tras realizar la función comprobamos cuantos nulos hemos podido eliminar.

In [22]:
cl.comprobar_nulos(df, "premiered")

3726

Podemos observar que hemos reducido drasticamente el numero de nulos. Anteriormente teníamos aproximadamente un 75% de nulos y hemos podido reducir esta cifra a 3726 que equivale al 17% respecto el total de filas 21725. Con esto hemos podido reducir aproximadamente el 77% de los nulos originales dandonos un muy buen resultado.

A continuación podemos observar unos ejemplos del resultado de estas operaciones.

In [23]:
df.sample(3)

,anime_id,name,score,genres,type,episodes,aired,premiered,status,studios,source,rating,rank,popularity,favorites,scored_by,nulos_fila,aired_start,aired_month
3464,4066,Tytania,6.80,"Adventure, Romance, Sci-Fi",TV,26,"Oct 9, 2008 to Mar 26, 2009",fall 2008,Finished,Artland,Light novel,PG-13 - Teens 13 or older,4827.0,4666,33,7151.0,0,2008,10
4920,7211,Rose O'Neill Kewpie,NaN,Comedy,TV,26,"Dec 2, 2009 to May 26, 2010",winter 2010,Finished,TMS Entertainment,Other,G - All Ages,19235.0,15782,0,NaN,2,2009,12
18003,44202,Fire Emblem Heroes - Book IV Ending Movie,5.68,"Action, Drama, Supernatural, Suspense",Special,1,"Nov 26, 2020",fall 2020,Finished,NaN,Game,PG-13 - Teens 13 or older,10158.0,18070,0,110.0,2,2020,11


Tras esto ya podemos eliminar la columna *aired* que no nos aporta información extra relevante y la columna *aired_month* que utilizamos para facilitar la implementaación. Además reordenamos las columnas para que *aired_start* no esté al final.

In [24]:
columnas = ['aired', 'aired_month']
df = cl.drop_columns(df, columnas)

In [25]:
columnas = [
    'anime_id',
    'name',
    'score',
    'genres',
    'type',
    'episodes',
    'aired_start',
    'premiered',
    'status',
    'studios',
    'source',
    'rating',
    'rank',
    'popularity',
    'favorites',
    'scored_by',
    'nulos_fila',
]
df = cl.ordenar_columnas(df, columnas)

In [26]:
df.sample()

,anime_id,name,score,genres,type,episodes,aired_start,premiered,status,studios,source,rating,rank,popularity,favorites,scored_by,nulos_fila
5968,9938,Ikoku Meiro no Croisée The Animation,7.42,Slice of Life,TV,12,2011,summer 2011,Finished,Satelight,Manga,G - All Ages,2020.0,2700,170,23689.0,0


##### Columna *source*

Convertimos la columna a tipo categórica

In [27]:
df['source'] = cl.convert_type(df, "source", "category")

##### Columna *rating*

Observamos como la columna de *rating* tiene valores dificiles de tratar. Vamos a simplificarlos:

In [28]:
df.rating.unique()

array(['R - 17+ (violence & profanity)', 'PG-13 - Teens 13 or older',
       'G - All Ages', 'PG - Children', 'R+ - Mild Nudity', nan,
       'Rx - Hentai'], dtype=object)

Con esta transformación consideramos que es más que suficiente para guardar los datos. Además, esta columna la podemos convertir a tipo categórica y previamente definiremos el orden de menor a mayor de los datos de *rating*.

In [29]:
df["rating"] = cl.parsear_rating(df)
df["rating"] = cl.convert_type(df, "rating", "category")

##### Columna *rank*

En esta columna vamos convertir los datos a valores numéricos de tipo **Int64** para poder tratar los valores nulos

In [30]:
df['rank'] = cl.convert_type(df, "rank", "Float64")
df['rank'] = cl.convert_type(df, "rank", "Int64")

##### Columna *scored_by*

De igual forma transformamos los datos de esta columna a valores numéricos

In [31]:
df['scored_by'] = cl.convert_type(df, "scored_by", "Float64")
df['scored_by'] = cl.convert_type(df, "scored_by", "Int64")

##### Tratamiento de nulos

Hemos observado como hay filas con numerosos nulos. Especialmente nos dimos cuenta que las filas con la columna *type* a nulo son datos con numerosos *NaN* y tras realizar una búsqueda en internet (al ser poco era viable) vimos como 2 de ellos eran comerciales publicitarios y por tanto los vamos a eliminar.

In [32]:
df[df["type"].isnull()]

,anime_id,name,score,genres,type,episodes,aired_start,premiered,status,studios,source,rating,rank,popularity,favorites,scored_by,nulos_fila
24764,55562,エデンズゼロ,NaN,"Action, Adventure, Fantasy, Supernatural",NaN,25,<NA>,NaN,Finished,NaN,Other,PG-13,0,0,0,<NA>,6
24532,55282,Nisshin Yakisoba U.F.O.: U.F.O. Red Lock-hen,NaN,NaN,NaN,1,2023,spring 2023,Finished,NaN,NaN,NaN,0,0,0,<NA>,8
24531,55281,Nisshin Homura Meshi: KonoSuba Bakuen-hen,NaN,NaN,NaN,1,2023,winter 2023,Finished,NaN,NaN,NaN,0,0,0,<NA>,8
24789,55590,les enfants de la baleines,NaN,NaN,NaN,12,2017,fall 2017,Finished,NaN,NaN,NaN,0,0,0,<NA>,8


In [33]:
df = cl.eliminar_nulos_cconcretos(df)

Tras esto ya tenemos finalizada la limpieza y podemos eliminar la columna auxiliar *nulos_fila*

In [34]:
df = cl.drop_columns(df, "nulos_fila")

##### Analisis final inicial

Con esto terminamos todo lo relevante de la limpieza. Todas estas modificaciones las hicimos sobre una copia del dataset original por lo que ahora crearemos un nuevo .csv con los cambios realizados.

In [43]:
df.sample(3)

,anime_id,name,score,genres,type,episodes,aired_start,premiered,status,studios,source,rating,rank,popularity,favorites,scored_by
21907,50612,Dr. Stone: Ryuusui,8.25,"Adventure, Comedy, Sci-Fi",Special,1,2022,summer 2022,Finished,TMS Entertainment,Manga,PG-13,295,1004,680,116119
5651,9203,K-On!!: Ura-On!!,6.54,Comedy,Special,9,2010,summer 2010,Finished,Kyoto Animation,Original,G,6131,3164,75,19789
10995,31646,3-gatsu no Lion,8.38,"Drama, Slice of Life",TV,22,2016,fall 2016,Finished,Shaft,Manga,PG-13,186,289,15123,270788


In [35]:
df.to_csv("../Proyecto1/data/anime_dataset_limpio.csv", index=False)